In [ ]:
# 引入第三方包
from rosbags.rosbag2 import Reader
from rosbags.serde import deserialize_cdr
import os, copy,traceback
import pandas as pd
import pickle
from tqdm import tqdm
from matplotlib import pyplot as plt

# 屏蔽warning
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# 数据合规性检查
from OmniScape_nav_lint import *
lint = OmniScapeNavLint()

from msgRegister import *

from coord_convert.transform import wgs2gcj, wgs2bd, gcj2wgs, gcj2bd, bd2wgs, bd2gcj

### 功能



### 使用说明

输入输出路径尽量放在固态盘下，实测固态盘和机械盘有16倍速度差距

OmniScape_nav_DB3解码工具（OmniScape_nav_DB3_parse.ipynb）为【HIT导航所智能感知信息处理团队-导航组】一站式数据源解析工具，适配导航车

Appendix 代码块为数据查询和说明功能，可注释不影响运行。

### 版本说明
| 版本       | 描述                                   |  时间 | 处理人 |
| --------- | -------------------------------------- |  -------- |-------- |
| v0.1      | 数据包读入，图片/文件导出，时间轴检查，话题名检查 |  2025-03-07 |李昕达 |

### 参数配置

#### topicFileMap 配置说明
| key       | 描述                                   |
| --------- | -------------------------------------- |
| type      | 类型，主要为IMU，GNSS，Base(真值)，Image，CompressedImage，Other类型 |
| topic     | 话题名，查询话题名运行代码块appendix.1 |
| filename  | 导出为csv的文件名                      |
| foldTopic | rosbag包话题有嵌套，这个列表负责展开这些嵌套，一般不需要动  |
| imgDir    | 仅Image，CompressedImage类型有效，图片路径，建议放固态盘  |


IMU，GNSS，Base(真值)的数量分别只能有一个，Image，CompressedImage导出图片，Other直出CSV


In [ ]:
# rosbag文件路径
DB3Filename = r'E:\NAV_DATASETS\CrossLoc\kc_htg'
topicFilePrefix = DB3Filename+"/toCSV/" # 转csv的目录 默认"./toCSV/"
imgFilePrefix = DB3Filename+"/toImg/" # 转图片的目录 默认"./toImg/"

lint.DB3Check(DB3Filename)
lint.dirExistCheck([topicFilePrefix,imgFilePrefix])

# 要转的topic
topicFileMap = [
    {"type":"IMU",              "topic":"/XSENSE6",                       "fileName":"XSENSE_IMU.csv",          "foldTopic":[]},
    {"type":"GNSS",             "topic":"/GNSS_0",                        "fileName":"GNSS_0.csv",               "foldTopic":["datahpposllh","datasat","datarawx"]},
    {"type":"Base",             "topic":"/CGI10103",                      "fileName":"CGI10103.csv",            "foldTopic":[]},
    # {"type":"Other",            "topic":"/livox/lidar",                   "fileName":"livox_lidar.csv",         "foldTopic":[]},
    # {"type":"Other",            "topic":"/livox/imu",                     "fileName":"livox_imu.csv",           "foldTopic":[]},
    # {"type":"Other",            "topic":"/UWB3",                          "fileName":"UWB3.csv",                "foldTopic":["data"]},
    # {"type":"Other",            "topic":"/UWB4",                          "fileName":"UWB4.csv",                "foldTopic":["data"]},
    # {"type":"Other",            "topic":"/UWB7",                          "fileName":"UWB7.csv",                "foldTopic":["data"]},
    {"type":"Other",            "topic":"/d435i/middle/imu",                     "fileName":"d435i_imu.csv",           "foldTopic":[]},
    {"type":"CompressedImage",  "topic":"/flir_camera/image_raw/compressed",    "imgDir":"flir_camera/"},
    {"type":"CompressedImage",  "topic":"/hk_camera/rgb/compressed",            "imgDir":"hk_camera/"},
    {"type":"CompressedImage", "topic":"/d435i/middle/color/image_raw/compressed", "imgDir":"d435i/color/"}, 
    # {"type":"CompressedImage", "topic":"/d435i/middle/infra1/image_rect_raw/compressed", "imgDir":"d435i/infra1/"},
    # {"type":"CompressedImage", "topic":"/d435i/middle/infra2/image_rect_raw/compressed", "imgDir":"d435i/infra2/"},
        # yi
    
    # {"type":"Other",            "topic":"/node2/uwb_nooploop_uwb_info_",        "fileName":"node2_info.csv",           "foldTopic":["data"]},
    # {"type":"Other",            "topic":"/node1/uwb_nooploop_uwb_info_",        "fileName":"node1_info.csv",           "foldTopic":["data"]},
    # {"type":"Other",            "topic":"/node1/mti_info_radxaMTi7Node",         "fileName":"node1_mti_info.csv",           "foldTopic":[]},
    # {"type":"Other",            "topic":"/node2/mti_info_radxaMTi7Node",         "fileName":"node2_mti_info.csv",           "foldTopic":[]},
    # {"type":"Other",            "topic":"/node1/mti_status_radxaMTi7Node",        "fileName":"node1_status.csv",           "foldTopic":[]},
    # {"type":"Other",            "topic":"/node2/mti_status_radxaMTi7Node",        "fileName":"node2_status.csv",           "foldTopic":[]},
]


# 根据topic自动配置msg
topic2msgDict = {}
with Reader(DB3Filename) as reader:
    for connection in reader.connections: topic2msgDict[connection.topic] = connection.msgtype
    
lint.topicMsgCheck(topicFileMap, topic2msgDict)
for topic in topicFileMap:
    if topic["topic"] in topic2msgDict: topic["msgType"] = topic2msgDict[topic["topic"]]

### 时间轴检查

In [ ]:
topicList = [x["topic"] for x in topicFileMap]
timeLine = {}
for topic in topicList:  timeLine[topic] = []
with Reader(DB3Filename) as reader:
    obj_connections = [x for x in reader.connections if x.topic in topicList]
    for connection, timestamp,_ in tqdm(reader.messages(connections=obj_connections), desc="时间轴解析中"):
        timeLine[connection.topic].append(timestamp)
lint = OmniScapeNavLint()
lint.timeLineCheck(topicList, timeLine)

### 卫星数量检查

In [ ]:
def gnssSysList2dict(gnssSysList):
    return {
        "GPS":gnssSysList.count("GPS"),
        "GAL":gnssSysList.count("GAL"),
        "BDS":gnssSysList.count("BDS"),
        "GLO":gnssSysList.count("GLO"),
        "ALL":len(gnssSysList)
            }
gnssid2SatSysDict = {0:"GPS",2:"GAL",3:"BDS",5:"?",6:"GLO"}
datarawxList = []
with Reader(DB3Filename) as reader: 
    obj_connections = [x for x in reader.connections if x.topic == "/GNSS_0"]
    for connection, timestamp, rawdata in tqdm(reader.messages(connections=obj_connections), desc="GNSS数据解析"):
        msg = deserialize_cdr(rawdata, connection.msgtype, typestore)
        datarawxList.append(msg.datarawx)
        
gnssidList = []
for datarawxItem in datarawxList:
    gnssidList.append(gnssSysList2dict([gnssid2SatSysDict[rawxBlock.gnssid] for rawxBlock in datarawxItem]))
    
df_gnssidList = pd.DataFrame(gnssidList)
df_gnssidList.plot(figsize=(12,3))
plt.show()

### 导出为图片

In [ ]:
imgtopicList = []
compressedtopicList = []
for topicDict in topicFileMap:
    if topicDict["type"] == "CompressedImage":        compressedtopicList.append(topicDict["topic"])
    elif topicDict["type"] == "Image":                imgtopicList.append(topicDict["topic"])
print(len(compressedtopicList))
if len(compressedtopicList) > 0:
    with Reader(DB3Filename) as reader:
        # CompressedImage
        obj_connections = [x for x in reader.connections if x.topic in compressedtopicList]
        for connection, timestamp, rawdata in tqdm(reader.messages(connections=obj_connections), desc="图片导出"):
            msg = deserialize_cdr(rawdata, connection.msgtype, typestore)
            [img,timestamp] = [msg.data,msg.header.stamp.sec+msg.header.stamp.nanosec/1e9]# 获取图片和时间戳
            imgName = msg.header.frame_id+" "+str(round(timestamp,6))# 获取图片名称
            # print(imgName)
            imgDir = ""
            for topicInfo in topicFileMap:
                if topicInfo["topic"] == connection.topic:
                    imgDir = topicInfo["imgDir"]
                    break
            os.makedirs(os.path.join(imgFilePrefix, imgDir), exist_ok=True)
            imgPath = os.path.join(imgFilePrefix, imgDir, imgName+".png")
            if not os.path.exists(imgPath):
                with open(imgPath, "wb") as f:   f.write(img)


### 导出为CSV

In [ ]:
for topicDict in topicFileMap:
    msgList = []
    if "fileName" not in topicDict: continue
    try:
        with Reader(DB3Filename) as reader:
            obj_connections = [x for x in reader.connections if x.topic == topicDict["topic"]]
            for connection, timestamp, rawdata in tqdm(reader.messages(connections=obj_connections), desc="CSV导出"+topicDict["topic"]):
                msg = deserialize_cdr(rawdata, connection.msgtype, typestore)
                dictMsg = copy.deepcopy(msg.__dict__)
                dictMsg['stamp_sec'] = msg.header.stamp.sec
                dictMsg['stamp_nanosec'] = msg.header.stamp.nanosec
                dictMsg['header'] = ""
                if len(topicDict["foldTopic"]) > 0:
                    for foldTopic in topicDict["foldTopic"]:
                        if foldTopic in dictMsg:
                            if len(dictMsg[foldTopic]) < 1: continue
                            keys = dictMsg[foldTopic][0].__dict__.keys()
                            for key in keys:
                                dictMsg[foldTopic+"_"+key] = [x.__getattribute__(key) for x in dictMsg[foldTopic]]
                                if len(dictMsg[foldTopic+"_"+key]) == 1:
                                    dictMsg[foldTopic+"_"+key] = dictMsg[foldTopic+"_"+key][0]
                        del dictMsg[foldTopic]
                msgList.append(dictMsg)
        csvFileName = os.path.join(topicFilePrefix, topicDict["fileName"])
        df = pd.DataFrame(msgList)
        df.to_csv(csvFileName, index=False)
    except Exception as e:
        traceback.print_exc();
        lint.logger.info(f"CSV导出失败  {topicDict['topic']}："+str(e))

#### 画出真值和GNSS的运动轨迹

In [ ]:
posTopicFileMap = [
    {"type":"GNSS",             "topic":"/GNSS_0",                        "fileName":"GNSS_0.csv",               "foldTopic":["datahpposllh","datasat","datarawx"]},
    {"type":"Base",             "topic":"/CGI10103",                      "fileName":"CGI10103.csv",            "foldTopic":[]},
]
llhGNSS = {"lat":[],"lon":[],"alt":[]}
llhBase = {"lat":[],"lon":[],"alt":[],"status_L":[], "status_H":[],"status_desc":[]}

s = 0.5
# matplotlib 中文
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

def bytes2hexstr(bytes):
    return ''.join(['%02X ' % b for b in bytes])

# GNSS
with Reader(DB3Filename) as reader: 
    obj_connections = [x for x in reader.connections if x.topic == posTopicFileMap[0]["topic"]]
    for connection, timestamp, rawdata in tqdm(reader.messages(connections=obj_connections), desc="GNSS数据解析"):
        # print(bytes2hexstr(rawdata[:]))
        msg = deserialize_cdr(rawdata, connection.msgtype, typestore)
        lat = (msg.datahpposllh[0].lat+msg.datahpposllh[0].lathp*1e-2)*1e-7
        lon = (msg.datahpposllh[0].lon+msg.datahpposllh[0].lonhp*1e-2)*1e-7
        alt = (msg.datahpposllh[0].height+msg.datahpposllh[0].heighthp*1e-2)*1e-7
        if lat > 40 and lat < 45.76 and lon >120 and lon <130:
            llhGNSS["lat"].append((msg.datahpposllh[0].lat+msg.datahpposllh[0].lathp*1e-2)*1e-7)
            llhGNSS["lon"].append((msg.datahpposllh[0].lon+msg.datahpposllh[0].lonhp*1e-2)*1e-7)
            llhGNSS["alt"].append((msg.datahpposllh[0].height+msg.datahpposllh[0].heighthp*1e-2)*1e-7)

def statusDescMap(H):
    LowDict = {        "0":"定位中",        "1":"粗对准",        "2":"精对准",        "3":"GNSS定位",        "4":"GNSS定向" ,
                "5":"RTK定位",        "6":"DMI组合",        "7":"DMI标定",        "8":"纯惯性",        "9":"零速校正",
                "A":"VG模式",        "B":"RTK定向",        "C":"初始化"}
    HighDict  = {        "0":"GNSS单点定位",        "2":"SBAS模式",        "4":"RTK固定解",        "5":"RTK浮点解"    }
    return HighDict[H]

# Base
with Reader(DB3Filename) as reader: 
    obj_connections = [x for x in reader.connections if x.topic == posTopicFileMap[1]["topic"]]
    for connection, timestamp, rawdata in tqdm(reader.messages(connections=obj_connections), desc="真值数据解析"):
        msg = deserialize_cdr(rawdata, connection.msgtype, typestore)
        if len(msg.fpd_gnsssta) > 1:
            llhBase["lat"].append(msg.fpd_gnsslatitude)
            llhBase["lon"].append(msg.fpd_gnsslongitude)
            llhBase["alt"].append(msg.fpd_gnssaltitude)
            llhBase["status_L"].append(str(msg.fpd_gnsssta)[-1])
            llhBase["status_H"].append(str(msg.fpd_gnsssta)[0])
            llhBase["status_desc"].append(statusDescMap(llhBase["status_H"][-1]))

'''
Low :0：定位中1：粗对准2：精对准3：GNSS定位4：GNSS定向5：RTK定位6：DMI组合7：DMI标定8：纯惯性9：零速校正A:VG模式B:RTK定向C：初始化
High :O:GNSS单点定位2：SBAS模式4：RTK固定解5：RTK浮点解
'''

# 定义状态对应的颜色映射
status_colors = {
    "GNSS单点定位": "red",
    "RTK固定解": "green",
    "RTK浮点解": "orange",
    # 可以根据需要添加其他状态的颜色
}

# plot
plt.figure(figsize=(6, 6))
plt.scatter(llhGNSS["lon"], llhGNSS["lat"], label="GNSS",s=s)

df = pd.DataFrame(llhBase)
# 根据状态分段绘制轨迹


for status in status_colors:
    mask = df['status_desc'] == status  # 假设状态描述存储在 'status_desc' 列中
    plt.scatter(df.loc[mask, "lon"], 
            df.loc[mask, "lat"], 
            color=status_colors[status],
            label=f"Base ({status})",s=s)

plt.legend()
plt.show()

In [ ]:
# 使用folium绘制轨迹
import folium
from folium.plugins import MarkerCluster
import branca.colormap as cm

llhBaseMod100 = {key: llhBase[key][::100] for key in llhBase}  # 每100个点取一个，减少数据量

# 创建地图
m = folium.Map(
            # tiles='https://webrd02.is.autonavi.com/appmaptile?lang=en&size=1&scale=1&style=8&x={x}&y={y}&z={z}',# 道路底图
            tiles='http://wprd02.is.autonavi.com/appmaptile?lang=zh_cn&size=1&style=6&x={x}&y={y}&z={z}', # 卫星底图
            attr='高德-中英文对照',
            control_scale=True,
            max_zoom=18,
            min_zoom=5,
            overlay=True,
            prefer_canvas=True,
            opacity=0.7,
            zoom_start=16)

# 定义状态对应的颜色映射
status_colors = {
    "GNSS单点定位": "red",
    "RTK固定解": "green",
    "RTK浮点解": "orange",
    # 可以根据需要添加其他状态的颜色
}

# 将llhGNSS数据添加到地图 - 使用散点图
for i in range(len(llhGNSS["lat"])):
    lon,lat = wgs2gcj(llhGNSS["lon"][i], llhGNSS["lat"][i])
    folium.CircleMarker(
        location=[lat, lon],
        radius=1,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.1,  # 降低透明度，使点更透明
        popup="GNSS点"
    ).add_to(m)

# 将llhBase数据按状态分组添加到地图
df = pd.DataFrame(llhBaseMod100)

# 根据状态分段绘制散点图
for status in status_colors:
    mask = df['status_desc'] == status
    if mask.any():  # 确保有匹配的数据
        status_df = df.loc[mask]
        print(len(status_df))
        for _, row in status_df.iterrows():
            lon,lat = wgs2gcj(row["lon"], row["lat"])
            folium.CircleMarker(
                location=[lat, lon],
                radius=1,
                color=status_colors[status],
                fill=True,
                fill_color=status_colors[status],
                fill_opacity=0.1,
                popup=f"Base点 ({status})"
            ).add_to(m)

# 添加图例
legend_html = '''
<div style="position: fixed;      bottom: 50px; right: 50px;  border:2px solid grey; z-index:9999; 
            background-color:white; padding: 10px;font-size:14px; ">
    <p><b>图例</b></p> <p><i class="fa fa-circle" style="color:blue"></i> GNSS</p>
'''

for status, color in status_colors.items():
    legend_html += f'<p><i class="fa fa-circle" style="color:{color}"></i> Base ({status})</p>'

legend_html += '</div>'

m.get_root().html.add_child(folium.Element(legend_html))

# 根据llhBase的范围调整地图视图
if not df.empty:
    # 计算llhBase数据的边界
    min_lat = df['lat'].min()
    max_lat = df['lat'].max()
    min_lon = df['lon'].min()
    max_lon = df['lon'].max()
    
    # 计算中心点
    center_lat = (min_lat + max_lat) / 2
    center_lon = (min_lon + max_lon) / 2
    
    # 设置地图中心点
    m.location = [center_lat, center_lon]
    
    # 计算适当的缩放级别
    # 添加一些边距
    lat_padding = (max_lat - min_lat) * 0.1
    lon_padding = (max_lon - min_lon) * 0.1
    
    # 调整地图边界
    m.fit_bounds([
        [min_lat - lat_padding, min_lon - lon_padding],
        [max_lat + lat_padding, max_lon + lon_padding]
    ])
    
    print(f"地图已调整到llhBase数据范围: 纬度({min_lat:.6f}~{max_lat:.6f}), 经度({min_lon:.6f}~{max_lon:.6f})")
else:
    print("警告: llhBase数据为空，无法调整地图视图")


# 设置初始缩放级别
m.zoom_start = 18


# 显示地图
m

In [ ]:
plt.figure(figsize=(18,4),dpi=200)
plt.plot(df['lat'].diff().tolist()[1:])

### 导出为*.ros_pickle

In [ ]:

rosPickle = {"IMU":[],"GNSS":[],"Base":[]}
rosPickleTopicDict = [x for x in topicFileMap if x["type"] in ["IMU","Base"]]
for topicDict in rosPickleTopicDict:
    msgList = []
    try:
        with Reader(DB3Filename) as reader:
            obj_connections = [x for x in reader.connections if x.topic == topicDict["topic"]]
            for connection, timestamp, rawdata in tqdm(reader.messages(connections=obj_connections), desc="ros pickle导出"+topicDict["topic"]):
                msg = deserialize_cdr(rawdata, connection.msgtype, typestore)
                dictMsg = copy.deepcopy(msg.__dict__)
                dictMsg['stamp_sec'] = msg.header.stamp.sec
                dictMsg['stamp_nanosec'] = msg.header.stamp.nanosec
                dictMsg['header'] = ""
                if len(topicDict["foldTopic"]) > 0:
                    for foldTopic in topicDict["foldTopic"]:
                        if foldTopic in dictMsg:
                            if len(dictMsg[foldTopic]) < 1: continue
                            keys = dictMsg[foldTopic][0].__dict__.keys()
                            for key in keys:
                                dictMsg[foldTopic+"_"+key] = [x.__getattribute__(key) for x in dictMsg[foldTopic]]
                                if len(dictMsg[foldTopic+"_"+key]) == 1:
                                    dictMsg[foldTopic+"_"+key] = dictMsg[foldTopic+"_"+key][0]
                        del dictMsg[foldTopic]
                msgList.append(dictMsg)
        for topicType in ["IMU","Base"]:
            if topicDict["type"] == topicType:
                rosPickle[topicType] = copy.deepcopy(msgList)
    except Exception as e:
        traceback.print_exc()
        lint.logger.info(f"ros pickle 导出失败  {topicDict['topic']}："+str(e))


### 单独适配GNSS

In [ ]:


rosPickleTopicDict = [x for x in topicFileMap if x["type"] in ["GNSS"]]
breakFlag = False
for topicDict in rosPickleTopicDict:
    msgList = []
    with Reader(DB3Filename) as reader:
        obj_connections = [x for x in reader.connections if x.topic == topicDict["topic"]]
        for connection, timestamp, rawdata in tqdm(reader.messages(connections=obj_connections), desc="ros pickle导出"+topicDict["topic"]):
            msg = deserialize_cdr(rawdata, connection.msgtype, typestore)
            dictMsg = copy.deepcopy(msg.__dict__)
            dictMsg['stamp_sec'] = msg.header.stamp.sec
            dictMsg['stamp_nanosec'] = msg.header.stamp.nanosec
            dictMsg['header'] = ""
            if len(topicDict["foldTopic"]) > 0:
                for foldTopic in topicDict["foldTopic"]:
                    if len(dictMsg[foldTopic]) < 1: continue
                    keys = dictMsg[foldTopic][0].__dict__.keys()
                    # print(foldTopic,keys)
                    # print(foldTopic,foldTopic == 'datasat')
                    # print(dictMsg.keys())
                    if foldTopic == 'datasat':
                        for key in keys:
                            # dictMsg[foldTopic+"_"+key] = [x.__getattribute__(key) for x in dictMsg[foldTopic]]
                            rawxSatMap = {}
                            # print(foldTopic,keys)
                            # print(dictMsg[foldTopic][0].__getattribute__(list(keys)[0]))
                            for epoch in dictMsg[foldTopic]:
                                satGnssId,satSvid = epoch.__getattribute__('gnssid'),epoch.__getattribute__('svid')
                                satElev,satAzim,satCn0 = epoch.__getattribute__('elev'),epoch.__getattribute__('azim'),epoch.__getattribute__('cno')
                                rawxSatMap["{:03d}-{:03d}".format(satGnssId,satSvid)] = {"elev":satElev,"azim":satAzim,"cn0":satCn0}
                                # print("{:03d}-{:03d}".format(satGnssId,satSvid), satGnssId, satSvid, satElev, satAzim, satCn0)
                            # if foldtopic == "datasat":
                            #     satGnssIdList,satSvidList,satElevList,satAzimList,satCn0List = 
                    else:
                        for key in keys:
                            # print(foldTopic+"_"+key)
                            dictMsg[foldTopic+"_"+key] = [x.__getattribute__(key) for x in dictMsg[foldTopic]]
                            if len(dictMsg[foldTopic+"_"+key]) == 1:
                                dictMsg[foldTopic+"_"+key] = dictMsg[foldTopic+"_"+key][0]
                    del dictMsg[foldTopic]
                    breakFlag = True
                    # break
            try:
                rawxGnssIdList,rawxSvidList = dictMsg["datarawx_gnssid"],dictMsg["datarawx_svid"]
                matchedElevList,matchedAzimList,matchedCn0List = [],[],[]
                for indexSat in range(len(dictMsg["datarawx_gnssid"])):
                    rawexGnssId,rawxSvid = dictMsg["datarawx_gnssid"][indexSat],dictMsg["datarawx_svid"][indexSat]
                    # print("{:03d}-{:03d}".format(rawexGnssId,rawxSvid))
                    # print(rawxSatMap["{:03d}-{:03d}".format(rawexGnssId,rawxSvid)])
                    matchedDict = rawxSatMap["{:03d}-{:03d}".format(rawexGnssId,rawxSvid)]
                    matchedElevList.append(matchedDict["elev"])
                    matchedAzimList.append(matchedDict["azim"])
                    matchedCn0List.append(matchedDict["cn0"])
                dictMsg["datasat_elev"] = matchedElevList
                dictMsg["datasat_azim"] = matchedAzimList
                dictMsg["datasat_cno"] = matchedCn0List
                dictMsg["datasat_numsvs"] = len(matchedElevList)
            except:
                dictMsg["datasat_elev"],dictMsg["datasat_azim"],dictMsg["datasat_cno"] = [],[],[]
                dictMsg["datarawx_gnssid"],dictMsg["datarawx_svid"] = [],[]
                dictMsg["datasat_numsvs"] = 0
            
            msgList.append(dictMsg)
for topicType in ["GNSS"]:
    if topicDict["type"] == topicType:
        rosPickle[topicType] = copy.deepcopy(msgList)

### 导出

In [ ]:
toImgPath = []
for topicFile in topicFileMap:
    if topicFile['type'] == "CompressedImage":
        toImgPath.append(topicFile)
    
rosPickle["toImgPath"] = toImgPath    
rosPickle["rosbagFilePath"] = DB3Filename
with open("name.ros_pickle","wb") as f:
    pickle.dump(rosPickle,f)

### Appendix.1
话题名查询

In [ ]:
count = 0
with Reader(DB3Filename) as reader:
    # topic and msgtype information is available on .connections list
    for connection in reader.connections:
        count += 1
        print(str(count)+"话题名："+connection.topic+" "*(60-len(connection.topic)), str(count)+"msg名："+connection.msgtype)